In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
df = pd.read_csv('../data/process_v1/process_data_6.csv')
df = df[df['city'] == 0]

In [3]:
df['is_street_house'] = (df['accessibility'] == 2).astype(np.int32)

In [4]:
df['landSize_ratio'] = df['landSize'] / df['acreage']

In [5]:
mean_land_size_df = df.groupby('administrative_genre')['landSize'].mean().reset_index().rename(columns = {'landSize': 'meanLandSize'})

df = df.merge(mean_land_size_df, how='left', on = 'administrative_genre')

df['landSize_ratio_with_administrative_genre'] = df['landSize'] / df['meanLandSize']
df['acreage_ratio_with_meanLandSize'] = df['acreage'] / df['meanLandSize']


del df['meanLandSize']

def partition_for_alley_width(alley_width):
    if math.isnan(alley_width):
        return np.nan
    if alley_width <= 2.5:
        return 1
    if alley_width <= 5:
        return 2
    if alley_width <= 7:
        return 3
    if alley_width <= 8:
        return 4

In [6]:
{'fitThreeCars': 0,
 'fitOneCarAndOneMotorbike': 1,
 'notInTheAlley': 2,
 'parkCar': 3,
 'theBottleNeckPoint': 4,
 'fitTwoCars': 5,
 'narrorRoad': 6}

{'fitThreeCars': 0,
 'fitOneCarAndOneMotorbike': 1,
 'notInTheAlley': 2,
 'parkCar': 3,
 'theBottleNeckPoint': 4,
 'fitTwoCars': 5,
 'narrorRoad': 6}

In [7]:
cat_cols = ['is_street_house', 'ward', 'street', 'district', 'typeOfRealEstate', 'houseDirection', 'accessibility', 'certificateOfLandUseRight'] + [c for c in df.columns.tolist() if '_lat' in c or '_lon' in c]
target_cols = ['target']
remove_cols = ['price', 'description', 'city'] + [c for c in df.columns if 'distance_hcm' in c]
time_cols = ['time']
num_cols = [c for c in df.columns.tolist() if c not in (cat_cols + target_cols + remove_cols + time_cols)]

cat_cols = list(set(cat_cols))
target_cols = list(set(target_cols))
remove_cols = list(set(remove_cols))
num_cols = list(set(num_cols))

print(len(cat_cols), len(target_cols), len(remove_cols), len(num_cols))

28 1 52 156


In [8]:
cat_cols

['nearest_3_lat',
 'accessibility',
 'nearest_0_lat',
 'nearest_7_lon',
 'typeOfRealEstate',
 'nearest_4_lat',
 'nearest_7_lat',
 'nearest_2_lon',
 'district',
 'houseDirection',
 'nearest_8_lon',
 'ward',
 'is_street_house',
 'nearest_8_lat',
 'nearest_5_lat',
 'nearest_6_lat',
 'nearest_0_lon',
 'nearest_4_lon',
 'nearest_5_lon',
 'nearest_2_lat',
 'street',
 'certificateOfLandUseRight',
 'district_lon',
 'nearest_3_lon',
 'nearest_6_lon',
 'nearest_1_lon',
 'nearest_1_lat',
 'district_lat']

In [9]:
num_cols

['endWidth',
 'distance_nearest_2',
 'nearest_8_district',
 'num_of_hospital_in_1000m_radius',
 'num_of_kindergarten_in_500m_radius',
 'distance_hn_Nguyễn Thị Minh Khai',
 'num_of_fuel_in_1000m_radius',
 'distance_hn_Đường Lê Anh Xuân, quận 1',
 'distance_nearest_0',
 'num_of_townhall - community_centre_in_500m_radius',
 'facade',
 'nearest_0_district',
 'num_of_fast_food_in_2000m_radius',
 'nearest_2_ward',
 'nearest_0_street',
 'num_of_cafe_in_2000m_radius',
 'num_of_marketplace_in_500m_radius',
 'acreage',
 'distance_hn_Công viên 30-4',
 'nearest_5_district',
 'nearest_5_street',
 'distance_nearest_1',
 'num_of_university_in_2000m_radius',
 'numberOfFloors',
 'nearest_4_street',
 'distance_hn_Nguyễn Tri Phương',
 'distance_hn_Đào Trí',
 'numberOfBathRooms',
 'distance_hn_Công viên khu đô thị Sala',
 'distance_hn_Công viên Gia Định',
 'nearest_3_ward',
 'landSize_ratio_with_administrative_genre',
 'frontWidth',
 'num_of_atm_in_500m_radius',
 'num_of_ward',
 'lat',
 'num_of_place_of_w

In [10]:
print(len(num_cols))

156


In [11]:
df[cat_cols] = df[cat_cols].fillna(100)
df[cat_cols] = df[cat_cols].astype(np.int32)

In [12]:
df = df[cat_cols + num_cols + time_cols + target_cols]

In [13]:
df = df[df[target_cols[0]] != np.inf]
df = df[df[target_cols[0]] != -np.inf]

In [14]:
df.shape

(49052, 186)

In [15]:
df = df[df[target_cols[0]] >= 10]
df = df[df[target_cols[0]] < 100 * 10]
# df = pd.concat([df[df['w'] < 50], df[df['w'].isnull()]])
# df = pd.concat([df[df['h'] < 50], df[df['w'].isnull()]])

In [16]:
df.to_csv('../data/process_v1/process_data_7_hcm.csv', index = False)